# CLIP-направляемая доменная адаптация StyleGAN-2


В ноутбуке реализуется упрощённый вариант метода StyleGAN-NADA (No-Data Domain Adaptation) для адаптации предобученного генератора StyleGAN-2 под новый визуальный домен (стиль sketch) без использования изображений целевого домена.

В качестве исходной модели используется предобученный генератор StyleGAN-2, обученный на датасете FFHQ. Для адаптации создаются две копии генератора: замороженная и обучаемая. Обучение проводится таким образом, чтобы для одного и того же латентного вектора изображения, сгенерированные обучаемой моделью, смещались в сторону целевого домена относительно изображений, полученных из замороженной модели.

Направление адаптации задаётся в пространстве CLIP через текстовые описания исходного и целевого доменов. Обучение осуществляется без дискриминатора и без использования изображений целевого домена, что соответствует постановке задачи no-data domain adaptation.

Все промежуточные результаты сохраняются для последующего анализа и продолжения экспериментов.

In [ ]:
import os, json, sys
from datetime import datetime
import importlib
import numpy as np
from PIL import Image
import torch
import torch.nn.functional as F
import copy
import csv
import glob
from PIL import Image
from contextlib import contextmanager
from tqdm.auto import tqdm

In [ ]:
!pip -q install ftfy regex tqdm pillow
!pip -q install git+https://github.com/openai/CLIP.git

%cd /content
if not os.path.exists("stylegan2-ada-pytorch"):
    !git clone -q https://github.com/NVlabs/stylegan2-ada-pytorch.git

sys.path.append("/content/stylegan2-ada-pytorch")


In [ ]:
import clip

# Исследование важности блоков

Ранние блоки StyleGAN-2 отвечают за пропороции лица, расположение основных частей. Средние - за более детальную структуру, форму глаз, носа, рта. Поздние блоки в основном контролируют контуры, текстуры, цвет, мелкие детали.

Так как стиль "скетч" не предполагает изменения пропорций лица, а влияет в первую очередь на линии, контуры и визуальную текстуру, попробуем разморозить только поздние блоки - последние 4 блока.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

BASE_DIR = "/content/drive/MyDrive/stylegan_nada_project"
RUN_NAME = "sketch_late_only_v0"
RUN_DIR = os.path.join(BASE_DIR, RUN_NAME)
CKPT_DIR = os.path.join(RUN_DIR, "checkpoints")
SAMPLES_DIR = os.path.join(RUN_DIR, "samples")
LOGS_DIR = os.path.join(RUN_DIR, "logs")

for d in [RUN_DIR, CKPT_DIR, SAMPLES_DIR, LOGS_DIR]:
    os.makedirs(d, exist_ok=True)

config = {
    "run_name": RUN_NAME,
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "source_prompt": "photo of a face",
    "target_prompt": "sketch portrait of a face",
    "size": 1024,
    "truncation": 0.7,
    "batch_size": 1,
    "max_steps": 500,
    "save_every": 50,
    "lr": 2e-4,
    "adam_betas": [0.0, 0.99],
    "late_k_blocks": 4,
    "fixed_seeds": [0, 1, 2, 3, 4, 5, 6, 7]
}

CONFIG_PATH = os.path.join(RUN_DIR, "config.json")
with open(CONFIG_PATH, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)


In [ ]:
PKL_PATH = "/content/ffhq.pkl"
if not os.path.exists(PKL_PATH):
    !wget -q -O /content/ffhq.pkl https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

In [ ]:
REPO_DIR = "/content/stylegan2-ada-pytorch"

assert os.path.isdir(REPO_DIR)
assert os.path.isdir(os.path.join(REPO_DIR, "dnnlib"))
assert os.path.isfile(os.path.join(REPO_DIR, "legacy.py"))

sys.path = [p for p in sys.path if p != REPO_DIR]
sys.path.insert(0, REPO_DIR)

for m in list(sys.modules.keys()):
    if m == "dnnlib" or m.startswith("dnnlib.") or m == "legacy":
        del sys.modules[m]

importlib.invalidate_caches()

import dnnlib
import legacy

Инициализация генератора и генерация baseline-изображений

In [ ]:
import warnings
warnings.filterwarnings("ignore", message=".*Failed to build CUDA kernels for upfirdn2d.*")
warnings.filterwarnings("ignore", message=".*Failed to build CUDA kernels for bias_act.*")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open(PKL_PATH, "rb") as f:
    net = legacy.load_network_pkl(f)

G_ema = net["G_ema"].to(device).eval()

def seed_to_z(seed: int, z_dim: int, device: torch.device) -> torch.Tensor:
    rnd = np.random.RandomState(seed)
    z = rnd.randn(1, z_dim).astype(np.float32)
    return torch.from_numpy(z).to(device)

@torch.no_grad()
def gen_img(G, z, truncation_psi: float):
    c = None
    img = G(z, c, truncation_psi=truncation_psi, noise_mode="const")
    img = (img * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    img = img[0].permute(1, 2, 0).cpu().numpy()
    return img

def make_grid(images, cols=8):
    rows = int(np.ceil(len(images) / cols))
    h, w, _ = images[0].shape
    grid = np.zeros((rows * h, cols * w, 3), dtype=np.uint8)
    for idx, im in enumerate(images):
        r = idx // cols
        c = idx % cols
        grid[r*h:(r+1)*h, c*w:(c+1)*w] = im
    return grid

baseline_imgs = []
for s in config["fixed_seeds"]:
    z = seed_to_z(s, G_ema.z_dim, device)
    baseline_imgs.append(gen_img(G_ema, z, truncation_psi=config["truncation"]))

baseline_grid = make_grid(baseline_imgs, cols=8)
baseline_path = os.path.join(SAMPLES_DIR, "baseline_frozen.png")
Image.fromarray(baseline_grid).save(baseline_path)

from IPython.display import display
display(Image.open(baseline_path))


## Загрузка CLIP и подготовка эмбеддингов текста

In [ ]:
clip_model, _ = clip.load("ViT-B/32", device=device, jit=False)
clip_model.eval()

CLIP_MEAN = torch.tensor([0.48145466, 0.4578275, 0.40821073], device=device).view(1,3,1,1)
CLIP_STD  = torch.tensor([0.26862954, 0.26130258, 0.27577711], device=device).view(1,3,1,1)

def to_clip_input(img_nchw_m1p1: torch.Tensor) -> torch.Tensor:
    x = (img_nchw_m1p1 + 1.0) / 2.0
    x = F.interpolate(x, size=(224, 224), mode="bilinear", align_corners=False)
    x = (x - CLIP_MEAN) / CLIP_STD
    return x

@torch.no_grad()
def clip_text_embed(text: str) -> torch.Tensor:
    tokens = clip.tokenize([text]).to(device)
    emb = clip_model.encode_text(tokens).float()
    emb = emb / emb.norm(dim=-1, keepdim=True)
    return emb[0]

def clip_image_embed(img_nchw_m1p1: torch.Tensor) -> torch.Tensor:
    x = to_clip_input(img_nchw_m1p1)
    emb = clip_model.encode_image(x).float()
    emb = emb / emb.norm(dim=-1, keepdim=True)
    return emb

t_src = clip_text_embed(config["source_prompt"])
t_tgt = clip_text_embed(config["target_prompt"])

d_txt = (t_tgt - t_src)
d_txt = d_txt / (d_txt.norm() + 1e-8)


## Подготовка обучаемой и замороженной копий генератора, разморозка последних блоков

In [ ]:
G_frozen = copy.deepcopy(G_ema).to(device).eval()
G_train  = copy.deepcopy(G_ema).to(device).train()

for p in G_train.parameters():
    p.requires_grad = False

k = int(config["late_k_blocks"])

resolutions = list(G_train.synthesis.block_resolutions)
late_resolutions = resolutions[-k:]

for r in late_resolutions:
    block = getattr(G_train.synthesis, f"b{r}")
    for p in block.parameters():
        p.requires_grad = True

for p in G_train.mapping.parameters():
    p.requires_grad = False

trainable = sum(p.numel() for p in G_train.parameters() if p.requires_grad)
total = sum(p.numel() for p in G_train.parameters())
trainable, total


## Оптимизатор, функция потерь

In [ ]:
train_params = [p for p in G_train.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(train_params, lr=config["lr"], betas=tuple(config["adam_betas"]))

def directional_clip_loss(img_train_m1p1: torch.Tensor, img_frozen_m1p1: torch.Tensor, d_txt: torch.Tensor) -> torch.Tensor:
    e_train = clip_image_embed(img_train_m1p1)
    e_froz  = clip_image_embed(img_frozen_m1p1)

    d_img = (e_train - e_froz)
    d_img = d_img / (d_img.norm(dim=-1, keepdim=True) + 1e-8)

    d_txt_b = d_txt.view(1, -1).expand_as(d_img)
    cos = (d_img * d_txt_b).sum(dim=-1)
    return (1.0 - cos).mean()

fixed_z = torch.cat([seed_to_z(s, 512, device) for s in config["fixed_seeds"]], dim=0)

LOSS_CSV = os.path.join(LOGS_DIR, "loss.csv")
if not os.path.exists(LOSS_CSV):
    with open(LOSS_CSV, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["step", "loss"])


## Обучение

In [ ]:
os.makedirs(CKPT_DIR, exist_ok=True)
os.makedirs(SAMPLES_DIR, exist_ok=True)

LOSS_CSV = os.path.join(RUN_DIR, "loss.csv")
if not os.path.exists(LOSS_CSV):
    with open(LOSS_CSV, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["step", "loss"])

max_steps = int(config["max_steps"])
save_every = int(config["save_every"])

In [ ]:
def latest_checkpoint_path():
    ckpts = sorted(glob.glob(os.path.join(CKPT_DIR, "checkpoint_*.pt")))
    return ckpts[-1] if ckpts else None

def save_checkpoint(step: int):
    path = os.path.join(CKPT_DIR, f"checkpoint_{step:06d}.pt")
    torch.save({
        "step": step,
        "G_train": G_train.state_dict(),
        "optimizer": optimizer.state_dict(),
        "config": config,
        "fixed_z": fixed_z.detach().cpu(),
    }, path)
    return path

def load_latest_checkpoint():
    ckpt_path = latest_checkpoint_path()
    if not ckpt_path:
        return 0
    ckpt = torch.load(ckpt_path, map_location="cpu")
    G_train.load_state_dict(ckpt["G_train"], strict=False)
    optimizer.load_state_dict(ckpt["optimizer"])
    return int(ckpt["step"]) + 1

start_step = load_latest_checkpoint()

In [ ]:
def to_uint8_hwc(img_nchw_m1p1: torch.Tensor) -> np.ndarray:
    x = (img_nchw_m1p1.clamp(-1, 1) + 1.0) / 2.0
    x = (x * 255.0).round().to(torch.uint8)
    x = x[0].permute(1, 2, 0).detach().cpu().numpy()
    return x


In [ ]:
@torch.no_grad()
def make_sample_grid(step: int):
    imgs_f = []
    imgs_t = []

    for i in range(fixed_z.shape[0]):
        z = fixed_z[i:i+1]

        img_f = G_frozen(z, None, truncation_psi=config["truncation"], noise_mode="const")
        img_t = G_train.eval()(z, None, truncation_psi=config["truncation"], noise_mode="const")

        imgs_f.append(to_uint8_hwc(img_f))
        imgs_t.append(to_uint8_hwc(img_t))

    G_train.train()

    grid_f = make_grid(imgs_f, cols=8)
    grid_t = make_grid(imgs_t, cols=8)
    grid = np.concatenate([grid_f, grid_t], axis=0)

    out_path = os.path.join(SAMPLES_DIR, f"sample_{step:06d}.png")
    Image.fromarray(grid).save(out_path)
    return out_path

In [ ]:
def train_one_step():
    z = torch.randn([config["batch_size"], G_train.z_dim], device=device)

    with torch.no_grad():
        img_frozen = G_frozen(z, None, truncation_psi=config["truncation"], noise_mode="const")

    img_train = G_train(z, None, truncation_psi=config["truncation"], noise_mode="const")

    loss = directional_clip_loss(img_train, img_frozen, d_txt)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    return float(loss.item())

train_one_step()

In [ ]:
@contextmanager
def suppress_output():
    devnull = open(os.devnull, "w")
    old_stdout, old_stderr = sys.stdout, sys.stderr
    try:
        sys.stdout, sys.stderr = devnull, devnull
        yield
    finally:
        sys.stdout, sys.stderr = old_stdout, old_stderr
        devnull.close()

In [ ]:
!pip -q install tqdm

In [ ]:
start_step = 0

for p in glob.glob(os.path.join(CKPT_DIR, "checkpoint_*.pt")):
    os.remove(p)

In [ ]:
print("start_step =", start_step)
print("max_steps  =", max_steps)
print("range length =", len(list(range(start_step, max_steps + 1))))


In [ ]:
log_every = int(config.get("log_every", 10))
preview_every = int(config.get("preview_every", save_every))

pbar = tqdm(range(start_step, max_steps + 1), desc="Обучение", leave=True)

for step in pbar:
    with suppress_output():
        loss_value = train_one_step()

    with open(LOSS_CSV, "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([step, loss_value])

    if step % log_every == 0 or step == start_step:
        pbar.set_postfix({"loss": f"{loss_value:.4f}"})

    if step % preview_every == 0 or step == max_steps:
        with suppress_output():
            out_path = make_sample_grid(step)
            save_checkpoint(step)

        img = Image.open(out_path)
        img.thumbnail((700, 700))
        display(img)


# Редактирование сгенерированных изображений

Для демонстрации редактирования сгенерированных изображений используются фиксированные латентные векторы z. Для каждого такого вектора сравниваются изображения, полученные из замороженной копии генератора (G_frozen) и обучаемой копии генератора (G_train). Это позволяет наглядно показать, как меняется визуальный стиль изображения при сохранении его структуры и идентичности. Таким образом, реализуется редактирование уже сгенерированных изображений, а не независимая генерация новых примеров.

In [ ]:
def latest_checkpoint_path():
    ckpts = sorted(glob.glob(os.path.join(CKPT_DIR, "checkpoint_*.pt")))
    return ckpts[-1] if ckpts else None

with open(PKL_PATH, "rb") as f:
    net = legacy.load_network_pkl(f)

G_ema = net["G_ema"].to(device).eval()

G_frozen = copy.deepcopy(G_ema).to(device).eval()
G_train  = copy.deepcopy(G_ema).to(device).eval()

ckpt_path = latest_checkpoint_path()

ckpt = torch.load(ckpt_path, map_location="cpu")
G_train.load_state_dict(ckpt["G_train"], strict=False)

fixed_z = ckpt["fixed_z"].to(device)

@torch.no_grad()
def make_pairs_grid(G_frozen, G_train, fixed_z, truncation_psi, out_path, cols=8, thumb=(900, 900)):
    imgs = []
    for i in range(fixed_z.shape[0]):
        z = fixed_z[i:i+1]

        img_f = G_frozen(z, None, truncation_psi=truncation_psi, noise_mode="const")
        img_t = G_train(z, None, truncation_psi=truncation_psi, noise_mode="const")

        imgs.append(to_uint8_hwc(img_f))
        imgs.append(to_uint8_hwc(img_t))

    grid = make_grid(imgs, cols=2)
    Image.fromarray(grid).save(out_path)

    im = Image.open(out_path)
    im.thumbnail(thumb)
    return im

out_path = os.path.join(SAMPLES_DIR, "edited_generated_images_from_checkpoint.png")
preview = make_pairs_grid(
    G_frozen=G_frozen,
    G_train=G_train,
    fixed_z=fixed_z,
    truncation_psi=config["truncation"],
    out_path=out_path,
    cols=8
)

display(preview)

### Анализ генераций, полученных на разных этапах обучения

In [ ]:
samples = sorted(glob.glob(os.path.join(SAMPLES_DIR, "sample_*.png")))

idx_middle = len(samples) // 2
idx_last = len(samples) - 1

selected = [
    ("Середина обучения", samples[idx_middle]),
    ("Конец обучения", samples[idx_last]),
]

for title, path in selected:
    img = Image.open(path)
    print(title)
    display(img)


**Вывод**

В данном эксперименте визуальное качество результатов улучшалось на протяжении всего процесса обучения, и наиболее выраженный скетч-эффект наблюдается на финальных шагах оптимизации. Разморозка 4-х последний блоков дала неплохой, но всё ещё слабый результат.

В качестве следующего шага планируется разморозить и обучить дополнительные слои, что позволит оценить их влияние на выраженность скетч-стиля.

# Обучение дополнительных слоёв

In [ ]:
# новые директории для эксперимента
EXP_TAG = "exp2_more_layers"
STAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

RUN_DIR_2 = os.path.join(os.path.dirname(RUN_DIR), f"{os.path.basename(RUN_DIR)}_{EXP_TAG}_{STAMP}")
CKPT_DIR_2 = os.path.join(RUN_DIR_2, "checkpoints")
SAMPLES_DIR_2 = os.path.join(RUN_DIR_2, "samples")

os.makedirs(CKPT_DIR_2, exist_ok=True)
os.makedirs(SAMPLES_DIR_2, exist_ok=True)

LOSS_CSV_2 = os.path.join(RUN_DIR_2, "loss.csv")
CONFIG_PATH_2 = os.path.join(RUN_DIR_2, "config.json")

with open(CONFIG_PATH_2, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

In [ ]:
CKPT_DIR = CKPT_DIR_2
SAMPLES_DIR = SAMPLES_DIR_2
LOSS_CSV = LOSS_CSV_2
CONFIG_PATH = CONFIG_PATH_2

В прошлом эксперименте мы обучали 4 последних слоя, на этот раз попробуем обучить 6 слоёв.

In [ ]:
k_new = 6
config["late_k_blocks"] = k_new

for p in G_train.parameters():
    p.requires_grad = False

resolutions = list(G_train.synthesis.block_resolutions)
late_resolutions = resolutions[-k_new:]

for r in late_resolutions:
    block = getattr(G_train.synthesis, f"b{r}")
    for p in block.parameters():
        p.requires_grad = True

for p in G_train.mapping.parameters():
    p.requires_grad = False

optimizer = torch.optim.Adam(
    [p for p in G_train.parameters() if p.requires_grad],
    lr=float(config["lr"]),
    betas=tuple(config["adam_betas"])
)

In [ ]:
log_every = int(config.get("log_every", 10))
preview_every = int(config.get("preview_every", save_every))

pbar = tqdm(range(start_step, max_steps + 1), desc="Обучение k=6", leave=True)

for step in pbar:
    with suppress_output():
        loss_value = train_one_step()

    with open(LOSS_CSV, "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([step, float(loss_value)])

    if step % log_every == 0 or step == start_step:
        pbar.set_postfix({"loss": f"{loss_value:.4f}"})

    if step % preview_every == 0 or step == max_steps:
        with suppress_output():
            out_path = make_sample_grid(step)
            save_checkpoint(step)

        img = Image.open(out_path)
        img.thumbnail((700, 700))
        display(img)

# Сравнение исходных изображений и результата обучения 6 последних блоков

In [ ]:
SAMPLES_DIR_K6 = "/content/drive/MyDrive/stylegan_nada_project/sketch_late_only_v0_exp2_more_layers_20260202_010304/samples"

samples_k6 = sorted(glob.glob(os.path.join(SAMPLES_DIR_K6, "sample_*.png")))
final_k6_path = samples_k6[-1]

display(Image.open(final_k6_path))


# Результаты обучения 4 слоёв vs 6 слоёв

In [ ]:
BASE_DIR = "/content/drive/MyDrive/stylegan_nada_project"

for d in sorted(os.listdir(BASE_DIR)):
    print(d)

In [ ]:
DIR_K4 = "/content/drive/MyDrive/stylegan_nada_project/sketch_late_only_v0/samples"
DIR_K6 = "/content/drive/MyDrive/stylegan_nada_project/sketch_late_only_v0_exp2_more_layers_20260202_010304/samples"

samples_k4 = sorted(glob.glob(os.path.join(DIR_K4, "sample_*.png")))
samples_k6 = sorted(glob.glob(os.path.join(DIR_K6, "sample_*.png")))

img_k4 = Image.open(samples_k4[-1])
img_k6 = Image.open(samples_k6[-1])

print("k = 4")
display(img_k4)
print("k = 6")
display(img_k6)


# Вывод

Как и ожидалось, после расширения числа обучаемых блоков с 4 до 6 визуальный эффект скетч-стилизации стал более выраженным.
Полученный результат можно считать приемлемым для поставленной задачи доменной адаптации, что подтверждает гипотезу о важности вклада не только поздних, но и части средних блоков генератора в перенос визуального стиля.

Для чистоты эксперимента проведём генерацию новых изображений, не использовавшихся в процессе обучения.
Для одних и тех же случайных латентных векторов сравним изображения, полученные из замороженной (G_frozen) и обучаемой (G_train) копий генератора.

In [ ]:
torch.manual_seed(123)

new_imgs_frozen = []
new_imgs_train = []

for _ in range(4):
    z = torch.randn([1, G_train.z_dim], device=device)

    with torch.no_grad():
        img_f = G_frozen(
            z, None,
            truncation_psi=config["truncation"],
            noise_mode="const"
        )
        img_t = G_train(
            z, None,
            truncation_psi=config["truncation"],
            noise_mode="const"
        )

    new_imgs_frozen.append(to_uint8_hwc(img_f))
    new_imgs_train.append(to_uint8_hwc(img_t))

print("G_frozen")
display(Image.fromarray(make_grid(new_imgs_frozen, cols=4)))

print("G_train (k = 6)")
display(Image.fromarray(make_grid(new_imgs_train, cols=4)))
